# Cultural Data Analysis

Introduction to working with datasets

In [ ]:
# import necessary libraries
import os, re, csv
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from itertools import islice
from nltk.corpus import stopwords
import spacy
import string
import pickle

## Loading the dataset: heritage homes webistes

The dataset is stored in a shared google drive:
https://drive.google.com/drive/folders/11Shm0edDOiWrOe56fzJQRZi-v_BPSW8E?usp=drive_link

Add it to your drive.

To access it, load your gdrive in 'Files' (see left pane of the notebook in google colab) and navigate to the shared folder. You may need to click on 'refresh' to make it appear on the list.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Country code: change here between 'NL' and 'UK'
cc = 'UK'

In [ ]:
gdrive_path = '/content/gdrive/MyDrive/CDA/'

In [ ]:
# open the data for one country (cc)
raw_data_file = gdrive_path+cc+'_dataset_website-content-crawler.json'

In [ ]:
# Import json data
df=pd.read_json(raw_data_file)

# Print the DataFrame
df.head()

In [ ]:
# check if there are further datasets to add per country

!ls "$gdrive_path" | grep 'UK'

In [ ]:
df_missing1 = pd.read_json(gdrive_path+'/UK_EH_dataset_website-content-crawler_2025-03-26_09-11-52-434.json')
df_missing2 = pd.read_json(gdrive_path+'/UK_NH_dataset_website-content-crawler_2025-03-26_16-28-44-248.json')
df_missing3 = pd.read_json(gdrive_path+'/UK_PC_dataset_website-content-crawler_2025-03-11_12-28-08-810.json')
result = pd.concat([df, df_missing1, df_missing2, df_missing3])
df = result
df.head()

In [ ]:
# select only two columns for analysis: url and text
df=df[['url','text']]
df.head()

Join all pages from a domain to an entry in the analysis. To do this, add a new column which will contain only the main domain name.

In [ ]:
# function to extract the main domain from the url in the dataset
def extract_main_domain(url):
    if not isinstance(str(url), str):
        print('NOT VALID',url)
        return None
    match = re.findall('(?:\w+\.)*\w+\.\w*', str(url)) #'www\.?([^/]+)'
    return match[0].lstrip('www.') if match else None

In [ ]:
# Load the list of domains from a csv file:
cc_column = cc+' domains'
#print(cc_column)

urls = pd.read_csv(gdrive_path+'url_lists/'+cc+'_urls.csv')[cc_column].values.tolist()

# Extract main domains from nl_urls
domains = {extract_main_domain(url) for url in urls if extract_main_domain(url) is not None}

# Check if main domains in list_of_links match any domain in nl_domains
matching_links = [link for link in df.url if extract_main_domain(link) in domains]

In [ ]:
# this cell can be skipped, it is only for verification

# check how many lines in the dataframe have a matching link to the list of urls
print(len(matching_links))

In [ ]:
# Add a new column 'domain' and fill it by applying the extract_main_domain function to the 'url' column
df['domain'] = df['url'].apply(extract_main_domain)
df.head()

## A. Scrape webpages: screenshots

Automatically make screenshots of all urls, and save them to your local drive for later analysis.

Code to do this is available elsewhere.

Analyze: https://medium.com/@sehjadkhoja0/title-exploring-and-analyzing-image-data-with-python-79a7f72f4d2b

In [ ]:
# Install necessary libraries
!pip install selenium webdriver-manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# set the output directory to be your current session's sample_data (will be lost when you close the window)
output_dir = '/content/sample_data/'

In [ ]:
# Install Playwright library
!pip install playwright
# Install Playwright's browser binaries (Chromium, Firefox, WebKit)
!playwright install

In [ ]:
import asyncio
from playwright.async_api import async_playwright

# Define the output directory
output_dir = '/content/sample_data/'

async def take_screenshots():
    async with async_playwright() as p:
        # Use Chromium as the browser
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        for i, url in enumerate(urls):
            try:
                print(f"Navigating to: {url}")
                await page.goto(url, wait_until='networkidle') # wait for network to be idle

                # Sanitize the URL to create a valid filename
                filename = f"screenshot_{i}_{url.replace('https://', '').replace('http://', '').replace('/', '_').replace('.', '_')}.png"
                filepath = os.path.join(output_dir, filename)

                # Take full-page screenshot
                await page.screenshot(path=filepath, full_page=True)
                print(f"Screenshot saved: {filepath}")
            except Exception as e:
                print(f"Error taking screenshot for {url}: {e}")
        await browser.close()
        print("Screenshot process completed.")

# Run the async function
await take_screenshots()

## B. Discourse Analysis

### B.1 Close reading one document
(we will consider one website as a 'document')

In [ ]:
# list all unique domain names
df['domain'].unique()

In [ ]:
# extract all rows (lines) where the value of 'domain' is 'lancashire.gov.uk'
df[df['domain'] == 'lancashire.gov.uk']

In [ ]:
# then combine these into a list of pages
document = df[df['domain'] == 'lancashire.gov.uk']['text'].tolist()

In [ ]:
document

In [ ]:
clean_document = []
for i in document:
  j = i.replace('\n', ' ').replace('\r', '')
  clean_document.append(j)

In [ ]:
clean_document

### B.2 Topic Modelling

In [ ]:
import sklearn.feature_extraction.text as text

# min_df: ignore words occurring in fewer than `n` documents
# stop_words: ignore very common words ("the", "and", "or", "to", ...)
vec = text.CountVectorizer(lowercase=True, min_df=100, stop_words='english')
dtm = vec.fit_transform(df['text'])

In [ ]:
print(f'Shape of document-term matrix: {dtm.shape}. '
      f'Number of tokens {dtm.sum()}')

In [ ]:
import sklearn.decomposition as decomposition
NUM_TOPICS = 10
lda_model = decomposition.LatentDirichletAllocation(
    n_components=NUM_TOPICS, learning_method='online', random_state=1)
lda_Z = lda_model.fit_transform(dtm)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)

In [ ]:
document_topic_distributions = lda_model.fit_transform(dtm)

In [ ]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vec.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

print("LDA Model:")
print_topics(lda_model, vec)
print("=" * 20)

In [ ]:
no_top_words = 12
no_top_documents = 5
lda_H = lda_model.components_
tf_feature_names = vec.get_feature_names_out()

def display_topics(H, Z, feature_names, docs, no_top_words, no_top_documents):
    for idx, topic in enumerate(H):
        print("Topic %d:" % (idx))
        print("KEYWORDS", " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( Z[:,
                                        idx] )[::-1][0:no_top_documents]
        # good for checking which documents are the most characteristic for certain topics
        for doc_index in top_doc_indices:
            print("TOP DOCS", docs[doc_index])

display_topics(lda_H, lda_Z, tf_feature_names, df['text'].tolist(), no_top_words, no_top_documents)

### B.3 Collocations

In [ ]:
# change 'UK' here for country code by hand
!wget 'https://raw.githubusercontent.com/jazoza/cultural-data-analysis/refs/heads/main/stopwords_archive/UK.txt'

In [ ]:
# load a list of 'stopwords' in the language you are analyzing
def get_stopwords_list(stop_file_path):
    """load stop words """
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))
stopwords_path = cc+".txt"
stopwords = get_stopwords_list(stopwords_path)

In [ ]:
special_stop_words = ['nbsp', ' ', '', '—', '\’s']
stopwords_ext = stopwords+special_stop_words

In [ ]:
#function to remove non-ascii characters
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

In [ ]:
#load spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
#function to clean and lemmatize comments
def clean_documents(text):
    #remove punctuations
    regex = re.compile('[' + re.escape(string.punctuation) + '\\r\\t\\n]')
    nopunct = regex.sub(" ", str(text))
    #use spacy to lemmatize comments
    doc = nlp(nopunct, disable=['parser','ner'])
    lemma = [token.lemma_ for token in doc]
    return lemma

In [ ]:
#apply function to clean and lemmatize comments
lemmatized = df.text.map(clean_documents)
#make sure to lowercase everything
lemmatized = lemmatized.map(lambda x: [word.lower() for word in x])
lemmatized.head()

In [ ]:
unlist_documents = [item for items in lemmatized for item in items]

In [ ]:
# You would use these commands to save lemmatized text into a 'pickle' for later use
# The current setup does not enable you to overwrite existing files in the CDA/jar folder,
# so you would have to save the 'pickle' files elsewhere (for example in sample_data folder)
# If you want to reactivate this code, remove the tripple quotes ''' from the beginning and end
'''
# save these outputs for later
with open(gdrive_path+'jar/lemmatized.pickle', 'wb') as handle_l:
    pickle.dump(lemmatized, handle_l, protocol=pickle.HIGHEST_PROTOCOL)

with open(gdrive_path+'jar/unlist_documents.pickle', 'wb') as handle_u:
    pickle.dump(unlist_documents, handle_u, protocol=pickle.HIGHEST_PROTOCOL)
  '''

In [ ]:
# load saved pickles
with open(gdrive_path+'jar/'+cc+'_lemmatized.pickle', 'rb') as handle_l:
    lemmatized = pickle.load(handle_l)

with open(gdrive_path+'jar/'+cc+'_unlist_documents.pickle', 'rb') as handle_u:
    unlist_documents = pickle.load(handle_u)

In [ ]:
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
# initiate bigrams and trigrams
bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()

In [ ]:
# identify all collocations in the flat list of words from all documents
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(unlist_documents)
trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(unlist_documents)

Calculate basic frequency

In [ ]:
bigram_freq = bigramFinder.ngram_fd.items()

In [ ]:
bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)

In [ ]:
bigramFreqTable.head().reset_index(drop=True)

In [ ]:
# compute basic trigrams frequency
trigram_freq = trigramFinder.ngram_fd.items()
trigramFreqTable = pd.DataFrame(list(trigram_freq), columns=['trigram','freq']).sort_values(by='freq', ascending=False)
trigramFreqTable[:10]

Find meaningful bi- and tri-grams by filtering adjectives and nouns based on an nltk functionality

In [ ]:
#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    for word in ngram:
        if word in stopwords_ext:
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

In [ ]:
#filter bigrams
filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda x: rightTypes(x))]

In [ ]:
filtered_bi[:10]

Use advanced statistical methods like the Chi-Square to identify meaninful collocations
https://en.wikipedia.org/wiki/Chi-squared_test

In [ ]:
# filter bigrams using chi-square
bigramChiTable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.chi_sq)), columns=['bigram','chi-sq']).sort_values(by='chi-sq', ascending=False)
bigramChiTable.head()

In [ ]:
# find meaningful trigrams by filtering basic frequency table
# function to filter trigrams
def rightTypesTri(ngram):
    if '-pron-' in ngram or '' in ngram or ' 'in ngram or '  ' in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in stopwords_ext:
            return False
    first_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    third_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in first_type and tags[2][1] in third_type:
        return True
    else:
        return False

In [ ]:
filtered_tri = trigramFreqTable[trigramFreqTable.trigram.map(lambda x: rightTypesTri(x))]
filtered_tri[:10]

In [ ]:
# Chi-sqare frequency calculation for trigrams
trigramChiTable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.chi_sq)), columns=['trigram','chi-sq']).sort_values(by='chi-sq', ascending=False)
trigramChiTable.head(20)

### B.4 Analyse specific collocations

In [ ]:
# search for words from this list or use another list
search_words = ['royal']

In [ ]:
# SCI-KIT method, produces lists of co-occurencies for specific terms
def vectorize_text(df):
    vectorizer = text.CountVectorizer()
    X = vectorizer.fit_transform(df['text'])
    return X, vectorizer

def find_collocations(text, target_words):
    words = text.split()
    collocations = []
    for i in range(len(words) - 1):
        if words[i] in target_words:
            collocations.append((words[i], words[i + 1]))
        if words[i + 1] in target_words:
            collocations.append((words[i + 1], words[i]))
    return collocations

def get_frequent_collocations(df, most_frequent_words):
    collocations = []
    for text in df['text']:
        collocations.extend(find_collocations(text, most_frequent_words))
    collocation_counts = Counter(collocations)
    frequent_collocations = {}
    for word in most_frequent_words:
        word_collocations = {collocation: count for collocation, count in collocation_counts.items() if word in collocation}
        frequent_collocations[word] = dict(islice(Counter(word_collocations).most_common(20), 20))
    return frequent_collocations

def analyze_word_collocations(df):
    X, vectorizer = vectorize_text(df)
    most_frequent_words = search_words
    frequent_collocations = get_frequent_collocations(df, most_frequent_words)
    return frequent_collocations

In [ ]:
collocations = analyze_word_collocations(df)

In [ ]:
data = []
for word, colloc_dict in collocations.items():
   for collocation, count in colloc_dict.items():
       #collocation_str = ' '.join(collocation)  # Join collocation words into a single string
       data.append([word, collocation[1], count])
collocations_df = pd.DataFrame(data, columns=['Word', 'Collocation', 'Count'])
print(collocations_df.to_markdown(index=True))

### B.5 Word2Vec model

In [ ]:
nltk.download('punkt_tab')

In [ ]:
from nltk.tokenize import word_tokenize

# X is a list of tokenized texts (i.e. list of lists of tokens)
X = [word_tokenize(item) for item in df.text.tolist()]
#print(X[0:3])
model = gensim.models.Word2Vec(X, min_count=6, vector_size=200) # min_count: how many times a word appears in the corpus; size: number of dimensions

In [ ]:
model.wv.most_similar(positive=["royal"], topn=12)

# Task
Install Selenium and WebDriver Manager, set up a headless Chrome browser, then iterate through the URLs in `gdrive_path+'url_lists/'+cc+'_urls.csv'`, navigate to each URL, take a full-page screenshot, and save the screenshots to a new folder named `screenshots` within the `gdrive_path` directory.

## Take Screenshots of URLs

### Subtask:
Install necessary libraries (Selenium, WebDriver Manager), set up a headless Chrome browser, iterate through the list of URLs, navigate to each, take a full-page screenshot, and save the screenshots to a dedicated folder within the specified Google Drive path.


## Summary:

### Data Analysis Key Findings
- The required libraries, Selenium and WebDriver Manager, were successfully installed.
- A headless Chrome browser environment was configured for automated web navigation.
- Full-page screenshots were successfully captured for each URL specified in the `_urls.csv` file.
- All generated screenshots were saved into a dedicated `screenshots` folder within the designated Google Drive path.

### Insights or Next Steps
- The captured screenshots are now available for subsequent visual inspection or automated analysis, which could include identifying content rendering issues or design inconsistencies.
- The next logical step involves reviewing these screenshots for anomalies or differences, potentially leading to further investigation of the content or layout of the captured web pages.
